In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink

# Read the CSV file
data = pd.read_csv("ALL_MICE.csv")

# Data preprocessing
data['Background'] = data['Background'].replace({'C57BL/6': 'C57', 'HET3': 'H3'})
data['Gender'] = data['Gender'].replace({'Male': 'M', 'Female': 'F'})
data['Batch'] = data['Batch'].str.extract(r'E_(\d{4}_\d+)_')[0]
data['Name Treatment'] = data['Name Treatment'].fillna('0')
data['Code'] = data['Code'].fillna('0')
data['Code'] = data['Code'] + '_' + data['Name Treatment']

# Define weight columns
weight_columns = [
    "1", "1.5", "2", "3", "4", "4.5", "5", "6", "8", "9", "10",
    "12", "14", "15", "16", "17", "18", "19", "20", "21", "22",
    "23", "24", "25", "26", "27", "28", "29", "30", "31", "32",
    "33", "34", "35", "36", "37", "38", "39", "40", "41", "42"
]
weight_columns_float = [float(x) for x in weight_columns]

# Group the data
grouped = data.groupby(["Background", "Gender", "Batch", "Code"])

# Create a mapping from checkboxes to group names
checkbox_to_group = {}
checkboxes = []
for name in grouped.groups.keys():
    cb = widgets.Checkbox(value=False, description=str(name), layout=widgets.Layout(width='600px'))
    checkboxes.append(cb)
    checkbox_to_group[cb] = name

# Create a scrollable container for the checkboxes
cb_container = widgets.VBox(checkboxes, layout={'overflow': 'scroll'})
display(cb_container)

# Create an output widget for the plot
output = widgets.Output()

# Update plot function
current_fig = None

def update_plot(change):
    global current_fig
    with output:
        clear_output(wait=True)
        current_fig, ax = plt.subplots(figsize=(12, 8))
        
        for cb in checkboxes:
            if cb.value:
                name = checkbox_to_group[cb]
                group = grouped.get_group(name)
                avg_weights = group[weight_columns].mean()
                
                # Determine the last valid age with data
                last_valid_age = avg_weights.last_valid_index()
                if last_valid_age is not None:
                    last_valid_index = weight_columns.index(last_valid_age)
                    valid_ages_float = weight_columns_float[:last_valid_index + 1]
                    avg_weights = avg_weights[:last_valid_index + 1]
                    
                    # Interpolate missing values within the range of actual data
                    interpolated_weights = avg_weights.interpolate(method='linear')
                    
                    # Calculate SEM only for the valid ages
                    sem_weights = group[weight_columns].sem()[:last_valid_index + 1]
                    
                    # Plot the data with error bars
                    ax.errorbar(valid_ages_float, interpolated_weights, yerr=sem_weights, label=str(name), fmt='-o', capsize=5)
        
        ax.set_xlabel('Age (months)')
        ax.set_ylabel('Average Body Weight (grams)')
        ax.set_title('Average Body Weight over Time with SEM')
        ax.legend(title='Group', bbox_to_anchor=(1.05, 1), loc='upper left')
        ax.set_xticks(weight_columns_float)
        ax.set_xticklabels(weight_columns, rotation='vertical')
        ax.set_ylim(0, 60)
        ax.grid(True)
        plt.tight_layout()
        plt.show()

# Button to update the plot
update_button = widgets.Button(description="Update Plot")
update_button.on_click(update_plot)

# Function to save the plot
def save_plot(b):
    global current_fig
    if current_fig:
        file_path = r"C:\Users\epite\OneDrive - EpiTERNA\Documentos\Desktop\EPITERNA Scripts\figure\figure.png"
        current_fig.savefig(file_path)
        print(f"Plot saved to {file_path}")
        
# Buttons for updating and saving the plot
update_button = widgets.Button(description="Update Plot")
update_button.on_click(update_plot)

save_button = widgets.Button(description="Save Plot")
save_button.on_click(save_plot)

# Displaying everything
display(widgets.HBox([update_button, save_button]), output)


Output()